## EXPLORATORY DATA ANALISYS

In [ ]:
# Colab library to upload files to notebook
from google.colab import files
 
# Install Kaggle library
!pip install -q kaggle
 
#library numerical python utk scientific computing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

In [ ]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [ ]:
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"tridessynatalia","key":"a07a5a9dc638adacef55d92e0f5b82e5"}'}

In [ ]:
!chmod 600 /content/.kaggle/kaggle.json

chmod: cannot access '/content/.kaggle/kaggle.json': No such file or directory


In [ ]:
# Downlaod data kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d amananandrai/ag-news-classification-dataset

kaggle.json
ag-news-classification-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip \*.zip

Archive:  ag-news-classification-dataset.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
root_path = '/content'
train_path = f'{root_path}/train.csv'
train_data = pd.read_csv(train_path)

test_path = f'{root_path}/test.csv'
test_data = pd.read_csv(test_path)
train_data.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


#### Menggabung kolom Title dan Description menjadi satu kolom bernama summary

In [ ]:
train_data['summary'] = train_data['Title'] + ' ' + train_data['Description']
test_data['summary'] = test_data['Title'] + ' ' + test_data['Description']

train_data = train_data.drop(columns=['Title', 'Description'])
test_data = test_data.drop(columns=['Title', 'Description'])

labels = {1:'World News', 2:'Sports News', 3:'Business News', 4:'Science-Technology News'}

train_data['label'] = train_data['Class Index'].map(labels)
test_data['label'] = test_data['Class Index'].map(labels)

In [ ]:
train_data = train_data.drop(columns=['Class Index'])
test_data = test_data.drop(columns=['Class Index'])
train_data.head()

,summary,label
0,Wall St. Bears Claw Back Into the Black (Reute...,Business News
1,Carlyle Looks Toward Commercial Aerospace (Reu...,Business News
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,Business News
3,Iraq Halts Oil Exports from Main Southern Pipe...,Business News
4,"Oil prices soar to all-time record, posing new...",Business News


In [ ]:
print(train_data.describe())
print(test_data.describe())

                                                  summary          label
count                                              120000         120000
unique                                             120000              4
top     Bristol plans to move 5,000 to open source Bri...  Business News
freq                                                    1          30000
                                                  summary          label
count                                                7600           7600
unique                                               7600              4
top     Pacific Oil Link Is Best for Russia, Ambassado...  Business News
freq                                                    1           1900


In [ ]:
test_data.describe()

,summary,label
count,7600,7600
unique,7600,4
top,"Pacific Oil Link Is Best for Russia, Ambassado...",Business News
freq,1,1900


#### Berdasarkan output diatas, dapat dilihat bahwa jumlah label sebanyak 4. Data train terdiri atas 120000 baris dan data test terdiri atas 7600 baris.

In [ ]:
print(train_data.info())
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   summary  120000 non-null  object
 1   label    120000 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7600 entries, 0 to 7599
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   summary  7600 non-null   object
 1   label    7600 non-null   object
dtypes: object(2)
memory usage: 118.9+ KB
None


In [ ]:
train_data.isna().sum()

summary    0
label      0
dtype: int64

In [ ]:
test_data.isna().sum()

summary    0
label      0
dtype: int64

#### Tidak terdapat nilai null pada dataset

## **TEXT PREPROCESSING**

### 1. Punctuation Removal

In [ ]:
import numpy as np 
import pandas as pd 
import re
import string

In [ ]:
def remove_punc(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

train_data['summary'] = train_data['summary'].apply(lambda x: remove_punc(x))
test_data['summary'] = test_data['summary'].apply(lambda x: remove_punc(x))

In [ ]:
print(train_data)
print(test_data)

                                                  summary          label
0       Wall St Bears Claw Back Into the Black Reuters...  Business News
1       Carlyle Looks Toward Commercial Aerospace Reut...  Business News
2       Oil and Economy Cloud Stocks Outlook Reuters R...  Business News
3       Iraq Halts Oil Exports from Main Southern Pipe...  Business News
4       Oil prices soar to alltime record posing new m...  Business News
...                                                   ...            ...
119995  Pakistans Musharraf Says Wont Quit as Army Chi...     World News
119996  Renteria signing a topshelf deal Red Sox gener...    Sports News
119997  Saban not going to Dolphins yet The Miami Dolp...    Sports News
119998  Todays NFL games PITTSBURGH at NY GIANTS Time ...    Sports News
119999  Nets get Carter from Raptors INDIANAPOLIS  All...    Sports News

[120000 rows x 2 columns]
                                                summary                    label
0     Fears for 

### 2. Case Folding

In [ ]:
import numpy as np 
import pandas as pd 

train_data['summary'] = train_data['summary'].str.lower()
train_data['label'] = train_data['label'].str.lower()

test_data['summary'] = test_data['summary'].str.lower()
test_data['label'] = test_data['label'].str.lower()

In [ ]:
print(train_data)
print(test_data)

                                                  summary          label
0       wall st bears claw back into the black reuters...  business news
1       carlyle looks toward commercial aerospace reut...  business news
2       oil and economy cloud stocks outlook reuters r...  business news
3       iraq halts oil exports from main southern pipe...  business news
4       oil prices soar to alltime record posing new m...  business news
...                                                   ...            ...
119995  pakistans musharraf says wont quit as army chi...     world news
119996  renteria signing a topshelf deal red sox gener...    sports news
119997  saban not going to dolphins yet the miami dolp...    sports news
119998  todays nfl games pittsburgh at ny giants time ...    sports news
119999  nets get carter from raptors indianapolis  all...    sports news

[120000 rows x 2 columns]
                                                summary                    label
0     fears for 

### 3. Tokenizing

In [ ]:
import nltk
nltk.download('punkt')
train_summary = train_data['summary']
test_summary = test_data['summary']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
word_tokens = []
for sent in train_summary:
    # print(word_tokenize(sent))
    word_tokens.append(word_tokenize(sent))

In [ ]:
word_tokens2 = []
for sent in test_summary:
    # print(word_tokenize2(sent))
    word_tokens2.append(word_tokenize(sent))

In [ ]:
train_data['summary'] = word_tokens
test_data['summary'] = word_tokens2

In [ ]:
print(train_data)
print(test_data)

                                                  summary          label
0       [wall, st, bears, claw, back, into, the, black...  business news
1       [carlyle, looks, toward, commercial, aerospace...  business news
2       [oil, and, economy, cloud, stocks, outlook, re...  business news
3       [iraq, halts, oil, exports, from, main, southe...  business news
4       [oil, prices, soar, to, alltime, record, posin...  business news
...                                                   ...            ...
119995  [pakistans, musharraf, says, wont, quit, as, a...     world news
119996  [renteria, signing, a, topshelf, deal, red, so...    sports news
119997  [saban, not, going, to, dolphins, yet, the, mi...    sports news
119998  [todays, nfl, games, pittsburgh, at, ny, giant...    sports news
119999  [nets, get, carter, from, raptors, indianapoli...    sports news

[120000 rows x 2 columns]
                                                summary                    label
0     [fears, fo

### 4. Stopwords Removal

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
train_summary = train_data['summary']
test_summary = test_data['summary']

for i in range(len(train_summary)):
    train_summary[i] = [w for w in train_summary[i] if w not in stopwords.words('english')]


for i in range(len(test_summary)):
    test_summary[i] = [w for w in test_summary[i] if w not in stopwords.words('english')]


In [ ]:
train_data['summary'] = train_summary
test_data['summary'] = test_summary

In [ ]:
print(train_data)
print(test_data)

                                                  summary          label
0       [wall, st, bears, claw, back, black, reuters, ...  business news
1       [carlyle, looks, toward, commercial, aerospace...  business news
2       [oil, economy, cloud, stocks, outlook, reuters...  business news
3       [iraq, halts, oil, exports, main, southern, pi...  business news
4       [oil, prices, soar, alltime, record, posing, n...  business news
...                                                   ...            ...
119995  [pakistans, musharraf, says, wont, quit, army,...     world news
119996  [renteria, signing, topshelf, deal, red, sox, ...    sports news
119997  [saban, going, dolphins, yet, miami, dolphins,...    sports news
119998  [todays, nfl, games, pittsburgh, ny, giants, t...    sports news
119999  [nets, get, carter, raptors, indianapolis, all...    sports news

[120000 rows x 2 columns]
                                                summary                    label
0     [fears, n,

### 5. Stemming

In [ ]:
import nltk
from nltk import PorterStemmer

In [ ]:
ps = PorterStemmer()
def stemming(tokenized_text):
  text = [ps.stem(word) for word in tokenized_text]
  return text 

In [ ]:
train_data['summary'] = train_data['summary'].apply(lambda x: stemming(x))
test_data['summary'] = test_data['summary'].apply(lambda x: stemming(x))
print(train_data.head())
print(test_data.head())

                                             summary          label
0  [wall, st, bear, claw, back, black, reuter, re...  business news
1  [carlyl, look, toward, commerci, aerospac, reu...  business news
2  [oil, economi, cloud, stock, outlook, reuter, ...  business news
3  [iraq, halt, oil, export, main, southern, pipe...  business news
4  [oil, price, soar, alltim, record, pose, new, ...  business news
                                             summary                    label
0  [fear, n, pension, talk, union, repres, worker...            business news
1  [race, second, privat, team, set, launch, date...  science-technology news
2  [ky, compani, win, grant, studi, peptid, ap, a...  science-technology news
3  [predict, unit, help, forecast, wildfir, ap, a...  science-technology news
4  [calif, aim, limit, farmrel, smog, ap, ap, sou...  science-technology news


### 6. Lemmatization

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk import word_tokenize, WordNetLemmatizer
wnl = nltk.WordNetLemmatizer()

In [ ]:
def lemmatize(tokenized_text):
    text = [wnl.lemmatize(word) for word in tokenized_text]
    return text

In [ ]:
train_data['summary'] = train_data['summary'].apply(lambda x: lemmatize(x))
test_data['summary'] = test_data['summary'].apply(lambda x: lemmatize(x))
print(train_data.head())
print(test_data.head())

                                             summary          label
0  [wall, st, bear, claw, back, black, reuter, re...  business news
1  [carlyl, look, toward, commerci, aerospac, reu...  business news
2  [oil, economi, cloud, stock, outlook, reuter, ...  business news
3  [iraq, halt, oil, export, main, southern, pipe...  business news
4  [oil, price, soar, alltim, record, pose, new, ...  business news
                                             summary                    label
0  [fear, n, pension, talk, union, repres, worker...            business news
1  [race, second, privat, team, set, launch, date...  science-technology news
2  [ky, compani, win, grant, studi, peptid, ap, a...  science-technology news
3  [predict, unit, help, forecast, wildfir, ap, a...  science-technology news
4  [calif, aim, limit, farmrel, smog, ap, ap, sou...  science-technology news


### Save Data Cleaned to Drive/Repository

In [ ]:
from google.colab import  drive

In [ ]:
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
train_data.to_csv('/drive/My Drive/train_data_cleaned.csv')

In [ ]:
test_data.to_csv('/drive/My Drive/test_data_cleaned.csv')

In [ ]:
train_data.to_excel('/drive/My Drive/train_data_cleaned.xlsx')

In [ ]:
test_data.to_excel('/drive/My Drive/test_data_cleaned.xlsx')